In [15]:
# 🤖 CYOA Story Generation - First ML Experiment

**Goal**: Test GPT-2 model for generating anime-inspired Choose Your Own Adventure responses

**ML Concepts We'll Learn**:
- **Pre-trained models**: Using AI that's already learned from millions of texts
- **Inference**: Getting the model to generate new text
- **Model evaluation**: Checking if the generated stories make sense
- **MLflow model tracking**: Logging model experiments alongside data experiments


SyntaxError: unterminated string literal (detected at line 5) (1652178399.py, line 5)

In [ ]:
# Import libraries
import pandas as pd
import mlflow
import mlflow.transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
import torch
import warnings
warnings.filterwarnings('ignore')

print("🚀 Libraries loaded successfully!")
print(f"📊 PyTorch version: {torch.__version__}")
print(f"🤖 Transformers library ready for ML magic!")


In [ ]:
## 📊 Step 1: Load Our CYOA Dataset

**ML Concept**: Always start with understanding your data before applying models.


In [ ]:
# Load our processed anime CYOA dataset
df = pd.read_csv('data/processed_story_dataset.csv')

print(f"📚 Dataset loaded: {len(df)} anime-inspired scenarios")
print(f"📝 Columns: {list(df.columns)}")
print("\n🎌 Sample prompt and response:")
print(f"Prompt: {df.iloc[0]['prompt']}")
print(f"Response: {df.iloc[0]['response']}")

# Display the full dataset for review
df


In [ ]:
## 🧠 Step 2: Set Up MLflow Experiment

**MLOps Concept**: Track every model experiment just like we tracked data experiments.


In [ ]:
# Create MLflow experiment for model tracking
experiment_name = "cyoa_model_experiments"
mlflow.set_experiment(experiment_name)

print(f"🎯 MLflow experiment created: {experiment_name}")
print("📊 This will track all our model experiments in one place")


In [ ]:
## 🤖 Step 3: Load Pre-trained GPT-2 Model

**ML Concept**: GPT-2 is a "language model" trained on millions of texts. We'll use it as-is first, then later teach it our anime style.


In [16]:
# Start MLflow run for this experiment
mlflow.start_run(run_name="gpt2_baseline_test")

# Log experiment parameters
mlflow.log_param("model_name", "gpt2")
mlflow.log_param("model_type", "pre_trained_baseline")
mlflow.log_param("dataset_size", len(df))
mlflow.log_param("max_length", 100)  # Max words to generate

print("🔄 Loading GPT-2 model (this might take a moment...)")

# Create a text generation pipeline (simplified interface)
# ML Concept: Pipeline = pre-processing + model + post-processing in one step
generator = pipeline(
    'text-generation',
    model='gpt2',  # Small GPT-2 model (117M parameters)
    tokenizer='gpt2',
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

print("✅ GPT-2 model loaded successfully!")
print(f"💾 Model size: ~117M parameters (that's 117 million learned patterns!)")
print(f"🖥️ Running on: {'GPU' if torch.cuda.is_available() else 'CPU'}")


🔄 Loading GPT-2 model (this might take a moment...)


Device set to use cpu


✅ GPT-2 model loaded successfully!
💾 Model size: ~117M parameters (that's 117 million learned patterns!)
🖥️ Running on: CPU


In [17]:
## 🎨 Step 4: Generate Stories from Our Prompts

**ML Concept**: "Inference" = using the trained model to generate new content.


SyntaxError: invalid syntax (4095306715.py, line 3)

In [ ]:
# Test story generation on our anime prompts
generated_stories = []

print("🎌 Generating anime-style CYOA responses...\n")

for i, row in df.iterrows():
    prompt = row['prompt']
    original_response = row['response']
    genre = row['genre']
    
    # Generate story continuation
    # ML Concept: We give the model a prompt and it predicts what comes next
    generated = generator(
        prompt,
        max_length=100,  # Total length including prompt
        num_return_sequences=1,  # Generate 1 story per prompt
        temperature=0.7,  # Creativity level (0=boring, 1=wild)
        do_sample=True,  # Enable creative sampling
        pad_token_id=generator.tokenizer.eos_token_id
    )
    
    # Extract just the generated part (remove original prompt)
    generated_text = generated[0]['generated_text']
    generated_response = generated_text[len(prompt):].strip()
    
    # Store results
    story_result = {
        'genre': genre,
        'prompt': prompt,
        'original_response': original_response,
        'generated_response': generated_response
    }
    generated_stories.append(story_result)
    
    # Display comparison
    print(f"🎭 {genre.upper()} GENRE:")
    print(f"📝 Prompt: {prompt}")
    print(f"✨ Original: {original_response}")
    print(f"🤖 Generated: {generated_response}")
    print("-" * 80)

print(f"🎉 Generated {len(generated_stories)} stories!")


In [ ]:
## 📊 Step 5: Evaluate and Log Results

**ML Concept**: Always measure how good your model's outputs are.


In [ ]:
# Simple evaluation metrics
total_stories = len(generated_stories)
avg_generated_length = sum(len(story['generated_response']) for story in generated_stories) / total_stories
avg_original_length = sum(len(story['original_response']) for story in generated_stories) / total_stories

# Log metrics to MLflow
mlflow.log_metric("total_stories_generated", total_stories)
mlflow.log_metric("avg_generated_length", avg_generated_length)
mlflow.log_metric("avg_original_length", avg_original_length)

# Save generated stories as artifact
results_df = pd.DataFrame(generated_stories)
results_df.to_csv('generated_stories_baseline.csv', index=False)
mlflow.log_artifact('generated_stories_baseline.csv')

print(f"📊 Evaluation Complete:")
print(f"📝 Stories generated: {total_stories}")
print(f"📏 Average generated length: {avg_generated_length:.1f} characters")
print(f"📏 Average original length: {avg_original_length:.1f} characters")
print(f"💾 Results saved to MLflow artifacts")

# Display results table
print("\n📋 Results Summary:")
results_df


In [ ]:
## 🎯 Step 6: Wrap Up & Next Steps

**Learning Questions to Consider**:
1. How do the generated stories compare to our original anime-inspired ones?
2. Does GPT-2 understand the CYOA format?
3. What anime elements are missing from the generated content?

**Next Experiments**:
- Fine-tune GPT-2 on our anime dataset
- Try different temperature settings for creativity
- Expand our dataset with more anime scenarios
- Add prompt engineering for better anime context


In [ ]:
# End MLflow run
mlflow.end_run()

print("🏁 Experiment complete!")
print("🌐 Check MLflow UI at http://localhost:5000 to see your model experiment")
print("📁 Look for the 'cyoa_model_experiments' experiment")
print("\n🚀 Ready for the next phase: Fine-tuning for anime storytelling!")
